In [1]:
import cv2
import torch
import pandas


In [2]:

# 모델 로드
model = torch.hub.load('ultralytics/yolov5', 'custom', 'yolov5x-seg.pt')


Using cache found in /Users/sanghee/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-7-4 Python-3.11.3 torch-2.0.1 CPU

Fusing layers... 
YOLOv5x-seg summary: 455 layers, 88774445 parameters, 0 gradients, 265.7 GFLOPs
WARNING ⚠️ YOLOv5 SegmentationModel is not yet AutoShape compatible. You will not be able to run inference with this model.


In [3]:
# 웹캠 열기
cap = cv2.VideoCapture(0)

In [4]:
while True:
    # Read the frame
    ret, frame = cap.read()

    # Preprocess the image
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (640, 640))
    img = torch.from_numpy(img).permute(2, 0, 1).float().div(255.0).unsqueeze(0)

    # Perform inference
    results = model(img)

    # Process the results
    for result in results.pred[0].detach():
        x1, y1, x2, y2, class_id, confidence = result.tolist()
        label = model.names[int(class_id)]
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        cv2.putText(frame, f'{label}: {confidence:.2f}', (int(x1), int(y1) - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Display the frame
    cv2.imshow('Webcam', frame)

    # Press 'q' to exit
    if cv2.waitKey(1) == ord('q'):
        break

# Release the webcam and close the window
cap.release()
cv2.destroyAllWindows()

AttributeError: 'list' object has no attribute 'xyxy'